# Data generation from nnrd to `.png` images and `.png` segmentation masks

In [1]:
import glob
from detectron.dataset_new_data import create_config_from_params, NrrdToTrainingTestSplitter, create_dataset

Set here the variable names for config generation

In [2]:
op_name = "20190917"
path_location = "crossval_full"
model_pth_name = "model_final_FIRST_INCISION_20190917.pth"
op_phase = "FIRST_INCISION"

In [3]:
config = create_config_from_params(op_name=op_name,
                                       path_location=path_location,
                                       op_phase=op_phase,
                                       train=False,
                                       save_img_and_mask=True)

Get the corresponding nrrds from `data/nrrd_to_process` folder by separating them into original images and annotations

In [8]:
folder_data = glob.glob(config['CREATE_DATASET']["FOLDER_DATA_NRRD"])
splitter_dataset_object = NrrdToTrainingTestSplitter()
images, mask = splitter_dataset_object.get_mask_images(folder_data)

11
11


Extracting images from nnrd and saving it to `path_location`/`op_name`/val (if `train=False` in `config`, else in `path_location`/`op_name`/train)

Extracting masks images from nnrd and saving it to `path_location`/`op_name`/masks_val (if `train=False` in `config`, else in `path_location`/`op_name`/masks).
The names corresponds to the XXXXXXXX_`op_phase`_image sequence number_layer_`layer_encoding`
`layer_encoding` - is the encoded as int name of the label, that is specified in `detectron/mapping_layers.py` (f.e for First incision it is `mapping_layers` dict.

By setting the `config["CREATE_DATASET"]['CREATE_DESCRIPTION_FILE']` parameter you specify whether the description `.json` file will be created, that is needed for training.

In [6]:
config["CREATE_DATASET"]['CREATE_DESCRIPTION_FILE'] = True
create_dataset(config=config, images=images[:1], mask=mask[:1])


NameError: name 'config' is not defined

# Training a network

Requirements for training a detectron model is having a dir with images and a descirption file.
The description file (f.e `dataset_registration_detectron2_multilabeled_all_layers.json`) represents the ground truth annotations of the images in the same dir.

Before the training starts this file will be registred by the `register_dataset_and_metadata()`

In [5]:
from detectron.detectron2_run import start_training
start_training(yaml_config=config,
                   train=True,
                   validation=False)


AssertionError: Dataset 'resection_train' is already registered!

# Validation

For the validation the weights `/detectron/output/crossval_full/20190917/` and the actual saved weights like `model_final_FIRST_INCISION_20190917.pth`
When the weights are loaded successfully, than
`taken ../detectron/output/crossval_full/20190917/model_final_FIRST_INCISION_20190917.pth` will be printed.

In [4]:
from detectron.detectron2_run import start_training
start_training(yaml_config=config,
                   train=False,
                   validation=True)

100%|██████████| 10/10 [00:01<00:00,  6.52it/s]


Took: ../data/output/crossval_full/20190917/train/20200504_FIRST_INCISION_00026.png
wrote to ./output/crossval_full/20190917/img/20200504_FIRST_INCISION_00026.png
Took: ../data/output/crossval_full/20190917/train/20200423_FIRST_INCISION_00036.png
wrote to ./output/crossval_full/20190917/img/20200423_FIRST_INCISION_00036.png
Took: ../data/output/crossval_full/20190917/train/20200416_FIRST_INCISION_00039.png
wrote to ./output/crossval_full/20190917/img/20200416_FIRST_INCISION_00039.png
Took: ../data/output/crossval_full/20190917/train/20200504_FIRST_INCISION_00035.png
wrote to ./output/crossval_full/20190917/img/20200504_FIRST_INCISION_00035.png
Took: ../data/output/crossval_full/20190917/train/20200622_FIRST_INCISION_00020.png
wrote to ./output/crossval_full/20190917/img/20200622_FIRST_INCISION_00020.png
Took: ../data/output/crossval_full/20190917/train/20200427_FIRST_INCISION_00041.png
wrote to ./output/crossval_full/20190917/img/20200427_FIRST_INCISION_00041.png
Took: ../data/output/c

IndexError: index 0 is out of bounds for dimension 0 with size 0

# Draw resection line based on the triangle points calculation

Here is performed the estimation of the resection line based on the predicted mask.

Please create the structure based on `/detectron/output/NAME_OP_TYPE/20190917/structure.md` **BEFORE** starting the resection line processing.


In [ ]:
# initialise the required parameters for resection line processing

dir_to_process = "crossval_full/20190917"
predicted_plane_list = sorted(glob.glob(
    f"./../output/{dir_to_process}/images_predicted_multilabeled/layerwise_masks/1/*.png"))

original_img_list = glob.glob(f"./../../data/output/{dir_to_process}/val/*.png")

mapping_dict = create_plane_img_mapping(
    predicted_plane_list, original_img_list)


Besides the image an `.csv` file is created which represents the quantitative statistics of the resection line:
- angle
- centroid
- end_point
- length of the line

In [ ]:
from detectron.plane_postprocessing.triangle_points import full_process_resection_line


full_process_resection_line(mapping_dict=mapping_dict,
                            dir_crossval=dir_to_process,
                            location_to_save=f'./../csv/detectron/plane_evaluation/artificial_resec_line/{dir_to_process[9:]}.json',
                            saving_fig=True)
